In [16]:
!pip install google-play-scraper
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
from google_play_scraper import app, reviews, Sort
import json

In [18]:
result, continuation_token = reviews('com.pandora.android',
                                    lang='en',
                                    country='us',
                                    sort = Sort.NEWEST,
                                    count = 532)

result, _ = reviews('com.pandora.android', 
                   continuation_token = continuation_token)

In [19]:
import pandas as pd
import numpy as np
import nltk

In [20]:
df = pd.DataFrame(result)
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,70833e90-e9ed-4cb0-89ab-34a65119728d,Jamie,https://play-lh.googleusercontent.com/a-/AFdZu...,Good,5,0,2205.1.1,2022-08-23 20:12:37,None,NaT
1,fe8c811a-9a1b-429c-bc7b-396cf461aea2,sherry murphy,https://play-lh.googleusercontent.com/a-/AFdZu...,Great playlist,5,0,2205.1.1,2022-08-23 19:47:44,None,NaT
2,c9e8316b-187a-4329-bd9d-329e06459a97,Skip NOLA,https://play-lh.googleusercontent.com/a-/AFdZu...,I met rock Rocky road Jesus Christ H8 ail supe...,2,0,2205.1.1,2022-08-23 19:05:47,None,NaT
3,639dd5a8-2e5c-4134-9306-41d3914affb8,Jose Leyva,https://play-lh.googleusercontent.com/a-/AFdZu...,Who ever had the stupid idea to remove the dro...,2,0,2205.1.1,2022-08-23 18:54:54,None,NaT
4,6e382433-447e-4ef3-a31f-64b2d6180452,Barry Nelson,https://play-lh.googleusercontent.com/a/AItbvm...,Listen all the time. A+,5,0,2205.1.1,2022-08-23 18:49:13,None,NaT


In [21]:
df = df.drop(columns=['userName',
                      'userImage',
                      'thumbsUpCount',
                      'reviewCreatedVersion',
                      'at',
                      'replyContent',
                      'repliedAt'])
df.head()

,reviewId,content,score
0,70833e90-e9ed-4cb0-89ab-34a65119728d,Good,5
1,fe8c811a-9a1b-429c-bc7b-396cf461aea2,Great playlist,5
2,c9e8316b-187a-4329-bd9d-329e06459a97,I met rock Rocky road Jesus Christ H8 ail supe...,2
3,639dd5a8-2e5c-4134-9306-41d3914affb8,Who ever had the stupid idea to remove the dro...,2
4,6e382433-447e-4ef3-a31f-64b2d6180452,Listen all the time. A+,5


In [22]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
df['score'] = scaler.fit_transform(df['score'].values.reshape(-1, 1))

In [23]:
df['score'] = np.where(df['score'] >= 0.5, 1, 0)

In [24]:
import nltk
 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
corpus = []
reviews = df['content']
for i in reviews:
    review = re.sub('[^a-zA-Z]', ' ',i)
    review = review.lower()
    review = review.split()

    ps = PorterStemmer()
 
    review = [ps.stem(word) for word in review
                if not word in set(stopwords.words('english'))]

    review = ' '.join(review) 
    corpus.append(review)

In [26]:
df.head()

,reviewId,content,score
0,70833e90-e9ed-4cb0-89ab-34a65119728d,Good,1
1,fe8c811a-9a1b-429c-bc7b-396cf461aea2,Great playlist,1
2,c9e8316b-187a-4329-bd9d-329e06459a97,I met rock Rocky road Jesus Christ H8 ail supe...,0
3,639dd5a8-2e5c-4134-9306-41d3914affb8,Who ever had the stupid idea to remove the dro...,0
4,6e382433-447e-4ef3-a31f-64b2d6180452,Listen all the time. A+,1


In [27]:
from sklearn.feature_extraction.text import CountVectorizer
 
dataset = df
    
cv = CountVectorizer(max_features = 1500)

X = cv.fit_transform(corpus).toarray()
 
y = dataset.iloc[:, 2].values

In [28]:
from sklearn.model_selection import train_test_split
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [29]:
from sklearn.ensemble import RandomForestClassifier
 
model = RandomForestClassifier(n_estimators = 501,
                            criterion = 'entropy')
                             
model.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=501)

In [30]:
RandomForestClassifier(criterion='entropy', n_estimators=501)

RandomForestClassifier(criterion='entropy', n_estimators=501)

In [31]:
y_pred = model.predict(X_test)
 
y_pred

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1])

In [32]:
from sklearn.metrics import confusion_matrix, accuracy_score
 
cm = confusion_matrix(y_test, y_pred)
ac = accuracy_score(y_test, y_pred)
print("Confusion Matrix: \n", cm,"\n", "Accuracy: ",  ac)

Confusion Matrix: 
 [[18 25]
 [ 5 85]] 
 Accuracy:  0.7744360902255639
